In [46]:
import os
import datetime
import csv

def get_lifelogs(api_key, api_url=os.getenv("LIMITLESS_API_URL") or "https://api.limitless.ai", endpoint="v1/lifelogs", limit=50, batch_size=10, includeMarkdown=True, includeHeadings=False, date=None, timezone=None, direction="asc"):
    all_lifelogs = []
    cursor = None

    # Default to today's date if not provided
    if date is None:
        date = datetime.now().strftime("%Y-%m-%d")
    print(date)
    # If limit is not None, enforce the batch size cap
    if limit is not None:
        batch_size = min(batch_size, limit)
    while True:
        params = {
            "limit": batch_size,
            "includeMarkdown": "true" if includeMarkdown else "false",
            "includeHeadings": "false" if includeHeadings else "true",
            "date": date,
            "direction": direction,
            "timezone": timezone if timezone else str(tzlocal.get_localzone())
        }
        
        if cursor:
            params["cursor"] = cursor
            
        response = requests.get(
            f"{api_url}/{endpoint}",
            headers={"X-API-Key": api_key},
            params=params,
        )

        if not response.ok:
            raise Exception(f"HTTP error! Status: {response.status_code}")

        data = response.json()
        lifelogs = data.get("data", {}).get("lifelogs", [])
        
        all_lifelogs.extend(lifelogs)
        
        if limit is not None and len(all_lifelogs) >= limit:
            return all_lifelogs[:limit]
        
        next_cursor = data.get("meta", {}).get("lifelogs", {}).get("nextCursor")
        
        if not next_cursor or len(lifelogs) < batch_size:
            break
        
        print(f"Fetched {len(lifelogs)} lifelogs, next cursor: {next_cursor}")
        cursor = next_cursor
    
    return all_lifelogs

def fetch_lifelogs_to_daily_csv(api_key, start_date, end_date, output_dir="lifelogs"):
    os.makedirs(output_dir, exist_ok=True)

    current_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        output_file = os.path.join(output_dir, f"{date_str}.csv")

        if os.path.exists(output_file):
            print(f"Skipped {date_str} (already exists)")
        else:
            try:
                daily_logs = get_lifelogs(api_key=api_key, limit=100, date=date_str)
                print(daily_logs)
                entries = []
                for log in daily_logs:
                    for entry in log.get("contents", []):
                        content = entry.get("content", "")
                        speaker = entry.get("speakerName", "null")
                        entries.append((content, speaker))

                if entries:
                    with open(output_file, "w", newline='', encoding="utf-8") as f:
                        writer = csv.writer(f)
                        writer.writerow(["content", "speakerName"])
                        writer.writerows(entries)
                    print(f"Saved {len(entries)} entries to {output_file}")
                else:
                    print(f"No entries for {date_str}")
            except Exception as e:
                print(f"Failed to fetch/save data for {date_str}: {e}")

        current_date += datetime.timedelta(days=1)


In [47]:
# Load environment variables from .env file
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(dotenv_path=Path("..") / ".env")

# Check if API key exists
api_key = os.getenv("LIMITLESS_API_KEY")
if not api_key:
    raise ValueError("LIMITLESS_API_KEY not found in environment variables. Please check your .env file.")

print("API key loaded successfully",api_key)
fetch_lifelogs_to_daily_csv(
    api_key=api_key,
    start_date="2025-04-17",
    end_date="2025-05-29"
)

API key loaded successfully 6920970b-0997-457a-ad7c-ef44d720c3ca
Skipped 2025-04-17 (already exists)
Skipped 2025-04-18 (already exists)
Skipped 2025-04-19 (already exists)
2025-04-20
[]
No entries for 2025-04-20
2025-04-21
[]
No entries for 2025-04-21
2025-04-22
[]
No entries for 2025-04-22
2025-04-23
[]
No entries for 2025-04-23
2025-04-24
[]
No entries for 2025-04-24
Skipped 2025-04-25 (already exists)
2025-04-26
[]
No entries for 2025-04-26
2025-04-27
[]
No entries for 2025-04-27
2025-04-28
[]
No entries for 2025-04-28
2025-04-29
[]
No entries for 2025-04-29
2025-04-30
[]
No entries for 2025-04-30
2025-05-01
[]
No entries for 2025-05-01
2025-05-02
[]
No entries for 2025-05-02
Skipped 2025-05-03 (already exists)
Skipped 2025-05-04 (already exists)
2025-05-05
[]
No entries for 2025-05-05
2025-05-06
[]
No entries for 2025-05-06
Skipped 2025-05-07 (already exists)
2025-05-08
[]
No entries for 2025-05-08
2025-05-09
[]
No entries for 2025-05-09
2025-05-10
[]
No entries for 2025-05-10
202